# DDSP IDつきで試験実装
### 構成
0. ライブラリのインストール
1. データのダウンロードと配置
2. 前処理
3. DDSPDeocderモジュール
4. MulchScaleloss
5. 実行

### 使用時の流れ
1. データローダーを用いてgoogle driveにデータセットをロードする（初回のみ）
2. 各wavファイルに対しあらかじめf0, loudnessを計算しgoogle driveに.ptファイルとして保存する
3. 計算を実行する

### 本モデルの機能
1. 任意の楽器の単旋律からf0(t), loudness(t)を抽出（当初librosaだったが、GPUを使って時短するという観点でtorchcrepeを用いる）
※今回はz(t)を入力としていない関係で、入力音声の時系列的変化（すなわちトランペットの吹き方の時系列変化）をとらえられない。声を入力とした場合、歌詞の情報は完全に排除された出力が得られる構成になっている

2. 抽出した情報からGRUを通して各時刻tに対し128次元の隠れ層状態を取得する。（最終出力は無視するため、_で受け取る）

3. 隠れ状態の情報を線形変換して倍音の構成(B,T,60)、全体音量(B,T,1)、ノイズ(B,T,ノイズの周波数バンド構成)を推定する




## 0. ライブラリのインストール

In [ ]:
# 必要なライブラリのインストール
!pip install torchcrepe librosa numpy matplotlib

In [ ]:
# ==========================================
# 1. ダウンロード不要！ 擬似マルチ楽器データ生成
# ==========================================
import os
import shutil
import glob
import torch
import numpy as np
import librosa

# 1. フォルダを一旦きれいにする (重要！)
OUTPUT_DIR = "data/features"
if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR) # フォルダごと削除
os.makedirs(OUTPUT_DIR, exist_ok=True) # 作り直し

print("フォルダを掃除しました。データを再生成します...")

# 設定
SAMPLE_RATE = 16000
HOP_LENGTH = 64
N_FFT = 2048
OUTPUT_DIR = "data/features"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def generate_synthetic_instrument(inst_name, wave_type, num_samples=5):
    print(f"Generating {inst_name} ({wave_type})...")

    for i in range(num_samples):
        duration = 5.0 # 5秒
        t = np.linspace(0, duration, int(duration * SAMPLE_RATE))

        # ピッチカーブ生成 (ドレミ...と動く)
        # ランダムな動きを加える
        base_freq = np.random.uniform(220, 440)
        freq_movement = np.sin(2 * np.pi * 0.5 * t) * 50 # ゆらぎ
        f0 = base_freq + freq_movement
        f0 = np.clip(f0, 50, 800) # 範囲制限

        # 位相計算
        phases = 2 * np.pi * np.cumsum(f0 / SAMPLE_RATE)

        # 波形生成
        if wave_type == 'sine':
            y = np.sin(phases)
        elif wave_type == 'sawtooth':
            y = (phases % (2 * np.pi)) / np.pi - 1.0
        elif wave_type == 'square':
            y = np.sign(np.sin(phases))

        # エンベロープ (音の強弱)
        env = 0.5 * (1.0 + np.sin(2 * np.pi * 1.0 * t)) # 1Hzでうねる
        y = y * env

        # ノイズを少し足す (リアルにするため)
        y += np.random.normal(0, 0.01, size=y.shape)
        y = y.astype(np.float32)

        # 特徴量抽出
        # f0 (Ground Truthを使えば高速だが、パイプライン確認のため抽出する)
        # 簡易的に計算上のf0を使う
        f0_tensor = torch.from_numpy(f0.astype(np.float32))

        # Loudness
        stft = librosa.stft(y, n_fft=N_FFT, hop_length=HOP_LENGTH)
        loudness = librosa.amplitude_to_db(np.abs(stft).mean(axis=0), ref=np.max).astype(np.float32)

        # サイズ合わせ
        min_len = min(len(f0_tensor), len(loudness))

        # ID割り当て
        if inst_name == 'Flute_Synth': inst_id = 0
        elif inst_name == 'Violin_Synth': inst_id = 1
        elif inst_name == 'Clarinet_Synth': inst_id = 2

        save_path = os.path.join(OUTPUT_DIR, f"{inst_name}_{i}.pt")

        torch.save({
            'audio': torch.from_numpy(y[:min_len*HOP_LENGTH]),
            'f0': f0_tensor[:min_len],
            'loudness': torch.from_numpy(loudness[:min_len]),
            'instrument_id': inst_id
        }, save_path)

# 実行：3種類の楽器を生成
# それぞれ 10個ずつファイルを生成します
generate_synthetic_instrument('Flute_Synth', 'sine', num_samples=10)     # ID=0
generate_synthetic_instrument('Violin_Synth', 'sawtooth', num_samples=10) # ID=1
generate_synthetic_instrument('Clarinet_Synth', 'square', num_samples=10) # ID=2

print("\n完了！ 'data/features' フォルダにデータを作成しました。")

フォルダを掃除しました。データを再生成します...
Generating Flute_Synth (sine)...
Generating Violin_Synth (sawtooth)...
Generating Clarinet_Synth (square)...

完了！ 'data/features' フォルダにデータを作成しました。


In [ ]:
import torch
import IPython.display as ipd
import numpy as np

def play_pt_file(file_path, sample_rate=16000):
    # 1. ファイルを読み込む
    try:
        data = torch.load(file_path)
    except FileNotFoundError:
        print(f"エラー: ファイルが見つかりません -> {file_path}")
        return

    # 2. 音声データを取り出す
    # 保存時のキー名が 'audio' である前提です
    if 'audio' not in data:
        print("エラー: このptファイルには 'audio' データが含まれていません。")
        print("含まれているキー:", data.keys())
        return

    audio_tensor = data['audio']

    # 3. 再生できる形式（Numpy配列）に変換
    # GPUに乗っている可能性や、形状が [1, T] になっている可能性を考慮
    audio_data = audio_tensor.detach().cpu().numpy().flatten()

    print(f"再生中: {file_path}")

    # 4. プレイヤーを表示
    ipd.display(ipd.Audio(audio_data, rate=sample_rate))

# --- 実行 ---
# 聞きたいファイルのパスを指定してください
# 例: 生成したシンセデータの1つを聞いてみる
play_pt_file("data/features/Violin_Synth_1.pt")
play_pt_file("data/features/Flute_Synth_9.pt")

再生中: data/features/Violin_Synth_1.pt


再生中: data/features/Flute_Synth_9.pt


## パラメータ設定

In [67]:
# ==========================================
# 0. 設定 (Hyperparameters)
# ==========================================
SAMPLE_RATE = 16000
N_FFT = 2048
HOP_LENGTH = 64  # 時間分解能を高めるため小さめに設定 (4ms)
BLOCK_SIZE = 512 # 1回の学習で扱うフレーム数

# シンセサイザー設定
N_HARMONICS = 60 # 倍音の数
N_NOISE_BANDS = 65 # ノイズフィルタの周波数バンド数

# モデル設定
HIDDEN_SIZE = 256
N_LAYERS = 1

# --- 設定 ---
OUTPUT_DIR = "data/features"

## 1. データのダウンロードとリバーブの除去



In [68]:

# 1. 必要なライブラリのインストール
!pip install -U demucs soundfile librosa

import torch
import torchaudio
import librosa
import soundfile as sf
import numpy as np
from demucs.pretrained import get_model
from demucs.apply import apply_model
import os


def separate_with_demucs():
    print("1. モデルをロード中 (htdemucs)...")
    device = "cuda" if torch.cuda.is_available() else "cpu"
    # Demucsの学習済みモデルを取得
    model = get_model('htdemucs').to(device)
    model.eval()

    print("2. 音源を準備中...")
    # 元音源を取得
    filename = librosa.ex('trumpet')
    # Demucsは 44.1kHz で動作するため、ここでリサンプリングして読み込む
    wav, sr = librosa.load(filename, sr=44100, mono=False)

    # モノラルの場合、ステレオ(2ch)に拡張する（モデルの仕様）
    if wav.ndim == 1:
        wav = np.stack([wav, wav]) # [2, T]

    # Tensor化: [Batch, Channels, Time]
    wav_tensor = torch.tensor(wav).unsqueeze(0).to(device)

    print("3. 分離処理を実行中...")
    # モデル適用 (shifts=1 は高速化のため。画質ならshifts=4推奨)
    with torch.no_grad():
        sources = apply_model(model, wav_tensor, shifts=1, split=True, overlap=0.25)

    # sourcesの形状: [Batch, Source_Count, Channels, Time]
    # htdemucsのソース順序: ["drums", "bass", "other", "vocals"]
    # トランペットのような主旋律は "vocals" に入ります
    vocals_idx = model.sources.index('vocals')
    vocals_wav = sources[0, vocals_idx].cpu().numpy() # [Channels, Time]

    # ステレオをモノラルに戻す（平均）
    vocals_mono = vocals_wav.mean(axis=0)

    print("4. 保存中...")
    # 学習用の 16kHz にリサンプリングして保存
    # (soundfileなどを使って手動リサンプリングするのは面倒なので、librosaで再ロード時にやってもいいですが
    #  ここでは簡易的なリサンプリングをして保存します)

    output_path = "dry_trumpet.wav"

    # 44.1k -> 16k へリサンプリング (librosaを使用)
    vocals_resampled = librosa.resample(vocals_mono, orig_sr=44100, target_sr=16000)

    # 保存 (soundfileを使用＝torchaudioのエラー回避)
    sf.write(output_path, vocals_resampled, 16000)

    print(f"成功！ '{output_path}' を作成しました。")

    # 確認再生
    from IPython.display import Audio, display
    display(Audio(output_path, rate=16000))

if __name__ == "__main__":
    separate_with_demucs()


1. モデルをロード中 (htdemucs)...
2. 音源を準備中...
3. 分離処理を実行中...
4. 保存中...
成功！ 'dry_trumpet.wav' を作成しました。


## 前処理

In [69]:
# ==========================================
# 1. ダウンロード不要！ 擬似マルチ楽器データ生成
# ==========================================
import os
import torch
import numpy as np
import librosa

# 設定
SAMPLE_RATE = 16000
HOP_LENGTH = 64
N_FFT = 2048
OUTPUT_DIR = "data/features"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def generate_synthetic_instrument(inst_name, wave_type, num_samples=5):
    print(f"Generating {inst_name} ({wave_type})...")

    for i in range(num_samples):
        duration = 5.0 # 5秒
        t = np.linspace(0, duration, int(duration * SAMPLE_RATE))

        # ピッチカーブ生成 (ドレミ...と動く)
        # ランダムな動きを加える
        base_freq = np.random.uniform(220, 440)
        freq_movement = np.sin(2 * np.pi * 0.5 * t) * 50 # ゆらぎ
        f0_high_res = base_freq + freq_movement
        f0_high_res = np.clip(f0_high_res, 50, 800) # 範囲制限

        # 位相計算
        phases = 2 * np.pi * np.cumsum(f0_high_res / SAMPLE_RATE)

        # 波形生成
        if wave_type == 'sine':
            y = np.sin(phases)
        elif wave_type == 'sawtooth':
            y = (phases % (2 * np.pi)) / np.pi - 1.0
        elif wave_type == 'square':
            y = np.sign(np.sin(phases))

        # エンベロープ (音の強弱)
        env = 0.5 * (1.0 + np.sin(2 * np.pi * 1.0 * t)) # 1Hzでうねる
        y = y * env

        # ノイズを少し足す (リアルにするため)
        y += np.random.normal(0, 0.01, size=y.shape)
        y = y.astype(np.float32) # float32は符号部が1ビット、小数点位置が8ビット、残り23ビットが数値情報

        # 特徴量抽出
        # f0 (Ground Truthを使えば高速だが、パイプライン確認のため抽出する)
        # 簡易的に計算上のf0を使う


        # Loudness: 上記で生成したf0について短時間フーリエ変換を行い、各時刻tについて全周波数の値の平均を求めたものをlibrosaライブラリのamplitude_to_db関数に通した出力をloudnessとする
        stft = librosa.stft(y, n_fft=N_FFT, hop_length=HOP_LENGTH)
        loudness = librosa.amplitude_to_db(np.abs(stft).mean(axis=0), ref=np.max).astype(np.float32)

        # 3. f0 (保存用) をダウンサンプリングする！
        # [::HOP_LENGTH] で「64個飛ばし」で取得します。これでLoudnessと同じ解像度になります。
        # (端数処理のため、Loudnessの長さに合わせてカットします)
        f0_frame = f0_high_res[::HOP_LENGTH]
        f0_tensor = torch.from_numpy(f0_frame.astype(np.float32))

        # サイズ合わせ
        min_len = min(len(f0_tensor), len(loudness))

        # ID割り当て
        if inst_name == 'Flute_Synth': inst_id = 0
        elif inst_name == 'Violin_Synth': inst_id = 1
        elif inst_name == 'Clarinet_Synth': inst_id = 2

        save_path = os.path.join(OUTPUT_DIR, f"{inst_name}_{i}.pt")

        torch.save({
            'audio': torch.from_numpy(y[:min_len*HOP_LENGTH]),
            'f0': f0_tensor[:min_len],
            'loudness': torch.from_numpy(loudness[:min_len]),
            'instrument_id': inst_id
        }, save_path)

# 実行：3種類の楽器を生成
# それぞれ 10個ずつファイルを生成します
generate_synthetic_instrument('Flute_Synth', 'sine', num_samples=10)     # ID=0
generate_synthetic_instrument('Violin_Synth', 'sawtooth', num_samples=10) # ID=1
generate_synthetic_instrument('Clarinet_Synth', 'square', num_samples=10) # ID=2

print("\n完了！ 'data/features' フォルダにデータを作成しました。")

Generating Flute_Synth (sine)...
Generating Violin_Synth (sawtooth)...
Generating Clarinet_Synth (square)...

完了！ 'data/features' フォルダにデータを作成しました。


## 4. モデル定義(HarmonySynthesizer, Noise_synthesizer, DDSP,LossF)

In [77]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
# ==========================================
# 1. 成功したコードのシンセサイザー (マスクなし版)
# ==========================================
class HarmonicSynthesizer(nn.Module):
    def __init__(self, sample_rate, hop_length):
        super().__init__()
        self.sample_rate = sample_rate
        self.hop_length = hop_length

    def forward(self, f0, harmonics_dist, global_amp):
        # フレームレートの音声を、16000Hzのサンプリングレートで音声生成するためにUpsampling
        f0_up = F.interpolate(f0.transpose(1, 2), scale_factor=self.hop_length, mode='linear').transpose(1, 2)
        amp_up = F.interpolate(global_amp.transpose(1, 2), scale_factor=self.hop_length, mode='linear').transpose(1, 2)
        harm_up = F.interpolate(harmonics_dist.transpose(1, 2), scale_factor=self.hop_length, mode='linear').transpose(1, 2)

        # Phase Calculation
        harmonic_indices = torch.arange(1, harmonics_dist.shape[-1] + 1, device=f0.device).float()
        frequencies = f0_up * harmonic_indices.view(1, 1, -1)
        phases = 2 * np.pi * torch.cumsum(frequencies / self.sample_rate, dim=1)

        # アンチエイリアシング（ナイキスト周波数（Sampling_rate/2）Hzを超える倍音は0にする）
        nyquist = self.sample_rate / 2.0

        # frequency < nyquist なら 1.0 (通す), そうでなければ 0.0 (消す)
        harmonic_freqs = f0_up * harmonic_indices.view(1, 1, -1)
        anti_aliasing_mask = (harmonic_freqs < nyquist).float()

        # Waveform Generation
        harmonic_waves = torch.sin(phases) * harm_up
        waveform = torch.sum(harmonic_waves, dim=-1, keepdim=True)

        return waveform * amp_up

# FilteredNoiseSynthesizer は前回のままでOK (変更なし)

class FilteredNoiseSynthesizer(nn.Module):
    def __init__(self, block_size, sample_rate):
        super().__init__()
        self.block_size = block_size # 周波数領域での処理単位

    def forward(self, filter_coeffs):
        """
        filter_coeffs: [B, T_frames, N_noise_bands] -> ノイズの周波数特性
        """
        B, T, N = filter_coeffs.shape
        # 時間領域でフィルタ係数をアップサンプリングするのは重いため、
        # 論文では「周波数領域での掛け算」として実装することが多いです。

        # 簡易実装:
        # 1. ホワイトノイズ生成 [B, T_samples]
        T_samples = T * HOP_LENGTH
        noise = torch.rand(B, T_samples, device=filter_coeffs.device) * 2 - 1

        # 2. フィルタ係数をアップサンプリングして適用
        # 本当はLTV(Linear Time Variant)フィルタですが、簡易的に振幅変調として実装します
        # (厳密なDDSPは周波数領域でWindowingをして畳み込みますが、ここでは直感的な実装にします)

        filter_up = F.interpolate(filter_coeffs.transpose(1, 2), scale_factor=HOP_LENGTH, mode='linear').transpose(1, 2)
        # フィルタ係数が周波数バンドごとのゲインを表すと仮定 (簡易化)
        # 本来はここからインパルス応答を作り、畳み込みます。

        # 今回は「学習可能なノイズエンベロープ」として、ノイズ成分全体を1つのバンドとみなして合成します
        # ※厳密な実装にはFFT畳み込みが必要です

        noise_gain = torch.mean(filter_up, dim=-1, keepdim=True) # 平均ゲイン
        return noise.unsqueeze(-1) * noise_gain


# ※クラス定義が必要なら前回のものをそのまま使ってください

# ==========================================
# 2. 成功したコードベースの Multi-Instrument DDSP
# ==========================================
class MultiInstrumentDDSP(nn.Module):
    def __init__(self, n_instruments, embed_dim=64):
        super().__init__()
        self.n_instruments = n_instruments
        self.embed_dim = embed_dim

        # ★追加: 楽器ID用 Embedding
        self.instrument_embedding = nn.Embedding(n_instruments, embed_dim)

        # Input: f0(1) + loudness(1) + embedding(64)
        input_dim = 2 + embed_dim

        # 成功したコードと同じ構造 (MLP -> GRU)
        self.mlp = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU()
        )
        self.gru = nn.GRU(256, HIDDEN_SIZE, num_layers=N_LAYERS, batch_first=True)

        # Output layers
        self.dense_amp = nn.Linear(HIDDEN_SIZE, 1)
        self.dense_harm = nn.Linear(HIDDEN_SIZE, N_HARMONICS)
        self.dense_noise = nn.Linear(HIDDEN_SIZE, N_NOISE_BANDS)

        self.harmonic_synth = HarmonicSynthesizer(SAMPLE_RATE, HOP_LENGTH)
        self.noise_synth = FilteredNoiseSynthesizer(BLOCK_SIZE, SAMPLE_RATE)

    def forward(self, f0_hz, f0_norm, loudness_norm, instrument_id):
        # 1. Embedding処理
        style_emb = self.instrument_embedding(instrument_id)
        T = f0_norm.shape[1]
        style_emb = style_emb.unsqueeze(1).expand(-1, T, -1)

        # 結合
        x = torch.cat([f0_norm, loudness_norm, style_emb], dim=-1)

        # 2. 本体処理 (成功コードと同じ)
        x = self.mlp(x)
        x, _ = self.gru(x)

        # 3. パラメータ射影 (成功コードと同じ exp を使用！)
        # バイアス初期化などの小細工もやめます（初期値のまま）
        amp = torch.exp(self.dense_amp(x))
        harm_dist = F.softmax(self.dense_harm(x), dim=-1)
        #noise_coeffs = torch.exp(self.dense_noise(x))

        # 4. 合成
        audio_harm = self.harmonic_synth(f0_hz, harm_dist, amp)
        #audio_noise = self.noise_synth(noise_coeffs)

        final_audio = audio_harm #+ audio_noise

        # ★重要: tanh は削除します！ 生の波形を返します。
        return final_audio.squeeze(-1)
# ==========================================
# 4. Loss Function (Multi-Scale Spectral Loss)
# ==========================================

def multiscale_spectral_loss(pred, target, fft_sizes=[2048, 1024, 512, 256, 128, 64]):
    loss = 0.0
    for n_fft in fft_sizes:
        hop = n_fft // 4

        # STFT
        pred_stft = torch.stft(pred, n_fft=n_fft, hop_length=hop, window=torch.hann_window(n_fft).to(pred.device), return_complex=True)
        target_stft = torch.stft(target, n_fft=n_fft, hop_length=hop, window=torch.hann_window(n_fft).to(target.device), return_complex=True)

        pred_mag = torch.abs(pred_stft)
        target_mag = torch.abs(target_stft)

        # Linear Loss (L1)
        loss += F.l1_loss(pred_mag, target_mag)

        # Log Loss (L1)
        loss += F.l1_loss(torch.log(pred_mag + 1e-7), torch.log(target_mag + 1e-7))

    return loss

## 3. データセットクラスの追加

In [78]:
# データセットクラスを追加。instrument_idを付与する
import glob
from torch.utils.data import Dataset, DataLoader

# Datasetクラス (globで全ファイルを読み込む)
class SyntheticDataset(Dataset):
    def __init__(self, data_dir, crop_len_sec=4.0, sample_rate=16000, hop_length=64):
        self.files = glob.glob(os.path.join(data_dir, "*.pt"))
        self.crop_len = int(crop_len_sec * sample_rate / hop_length)
        self.hop_length = hop_length
        self.f_min, self.f_max = 0.0, 2000.0
        self.l_min, self.l_max = -120.0, 0.0
        print(f"Dataset Loaded: {len(self.files)} files found.")

    def __len__(self):
        return 1000

    def __getitem__(self, idx):
        path = np.random.choice(self.files)
        data = torch.load(path)

        audio = data['audio'] #サンプリングレート
        f0 = data['f0'] #フレームレート
        loud = data['loudness'] #フレームレート
        inst_id = data['instrument_id'] # 保存されたID (0, 1, 2)

        # Random Crop
        total_frames = len(f0)
        if total_frames > self.crop_len:
            start = np.random.randint(0, total_frames - self.crop_len)
        else:
            start = 0
        end = start + self.crop_len

        f0 = f0[start:end]
        loud = loud[start:end]
        audio_crop = audio[start*self.hop_length : end*self.hop_length]

        # 正規化
        f0_norm = (f0 - self.f_min) / (self.f_max - self.f_min + 1e-7)
        loud_norm = (loud - self.l_min) / (self.l_max - self.l_min + 1e-7)
        loud_norm = torch.clamp(loud_norm, 0.0, 1.0)

        # Padding if needed
        if len(f0) < self.crop_len:
            pad = self.crop_len - len(f0)
            f0 = F.pad(f0, (0, pad))
            f0_norm = F.pad(f0_norm, (0, pad))
            loud_norm = F.pad(loud_norm, (0, pad))
            audio_crop = F.pad(audio_crop, (0, pad * self.hop_length))

        return {
            'f0': f0.unsqueeze(-1),
            'f0_norm': f0_norm.unsqueeze(-1),
            'loudness_norm': loud_norm.unsqueeze(-1),
            'instrument_id': torch.tensor(inst_id, dtype=torch.long),
            'audio': audio_crop
        }

## 4. 学習の実行

In [79]:


# --- 前回のDDSPモデル定義など (省略せずに必要なクラスを再定義) ---
# ※ ここに前回の回答にある「HarmonicSynthesizer」「FilteredNoiseSynthesizer」「DDSP」クラスがある前提です。
#    長くなるため、クラス定義部分は前回のコードをそのまま貼り付けてください。
#    (以下は学習ループのメイン部分のみ記述します)


# ==========================================
# 学習実行
# ==========================================
def train_synthetic():
    dataset = SyntheticDataset(OUTPUT_DIR)
    dataloader = DataLoader(dataset, batch_size=16, shuffle=True)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # 楽器は3種類 (Sine, Saw, Square)
    # モデルは前回の「成功したコードベースのMultiInstrumentDDSP」を使用してください
    model = MultiInstrumentDDSP(n_instruments=3, embed_dim=64).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    # ★追加: スケジューラ (Lossが下がらなくなったら学習率を半分にする)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=5)

    print("Synthetic Training Start...")

    for epoch in range(1, 101): # テストなので20エポックで十分
        total_loss = 0
        for i, batch in enumerate(dataloader):
            f0_hz = batch['f0'].to(device)

            f0_norm = batch['f0_norm'].to(device)
            loud_norm = batch['loudness_norm'].to(device)
            inst_id = batch['instrument_id'].to(device)
            target = batch['audio'].to(device)

            optimizer.zero_grad() # 勾配を初期化。初期化しないと、前回のepochでパラメータ更新に使われた勾配が残ってしまい、学習の妨げになってしまう

            # tanhなし、exp使用のモデル
            pred = model(f0_hz, f0_norm, loud_norm, inst_id)
            loss = multiscale_spectral_loss(pred, target)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            total_loss += loss.item()
            avg_loss = total_loss / (i + 1)

            if i % 10 == 0:
                print(f"Epoch {epoch} | Step {i} | Loss: {loss.item():.4f}")

        # エポック全体の平均lossを計算
        avg_loss = total_loss / len(dataloader)
        print(f"=== Epoch {epoch} Finished | Avg Loss: {avg_loss:.4f} ===")

        # スケジューラに報告
        scheduler.step(total_loss)

        # モデルを保存
        torch.save(model.state_dict(), f"ddsp_multi_synth_{epoch}.pth")

    torch.save(model.state_dict(), "ddsp_multi_synth_final.pth")
    print("All training finished. Model saved.")

if __name__ == "__main__":
    train_synthetic()

Dataset Loaded: 30 files found.
Synthetic Training Start...
Epoch 1 | Step 0 | Loss: 18.6441
Epoch 1 | Step 10 | Loss: 14.0611
Epoch 1 | Step 20 | Loss: 13.2761
Epoch 1 | Step 30 | Loss: 12.5238
Epoch 1 | Step 40 | Loss: 11.3567
Epoch 1 | Step 50 | Loss: 11.5504
Epoch 1 | Step 60 | Loss: 12.6128
=== Epoch 1 Finished | Avg Loss: 13.4864 ===
Epoch 2 | Step 0 | Loss: 13.5451
Epoch 2 | Step 10 | Loss: 11.4019
Epoch 2 | Step 20 | Loss: 13.3198
Epoch 2 | Step 30 | Loss: 12.2759
Epoch 2 | Step 40 | Loss: 12.0009
Epoch 2 | Step 50 | Loss: 12.4848
Epoch 2 | Step 60 | Loss: 12.9678
=== Epoch 2 Finished | Avg Loss: 12.4587 ===
Epoch 3 | Step 0 | Loss: 12.4231
Epoch 3 | Step 10 | Loss: 11.9657
Epoch 3 | Step 20 | Loss: 10.9369
Epoch 3 | Step 30 | Loss: 17.2137
Epoch 3 | Step 40 | Loss: 25.5087
Epoch 3 | Step 50 | Loss: 16.1265
Epoch 3 | Step 60 | Loss: 12.1242
=== Epoch 3 Finished | Avg Loss: 12.4080 ===
Epoch 4 | Step 0 | Loss: 12.9989
Epoch 4 | Step 10 | Loss: 11.8716
Epoch 4 | Step 20 | Loss: 1

## 5. 音の確認用コード

In [76]:
def listen_current_model(model):
    print("▼▼▼ Epoch 10 時点の音色チェック ▼▼▼")

    # 3種類の音色をテスト (0:Flute/Sine, 1:Violin/Saw, 2:Clarinet/Square)
    test_ids = [0, 1, 2]
    inst_names = ["Flute (Sine)", "Violin (Saw)", "Clarinet (Square)"]

    # テスト用入力データの作成 (ド・ミ・ソ...と動く)
    duration = 4.0
    n_frames = int(duration * SAMPLE_RATE / HOP_LENGTH)
    t = np.linspace(0, duration, n_frames, endpoint=False)

    # 音程: 220Hzから始まり、1秒ごとに倍音列を登る
    f0 = 220.0 * (1 + np.floor(t)) # 220, 440, 660, 880
    f0 = f0.astype(np.float32)
    f0_tensor = torch.from_numpy(f0).view(1, -1, 1).to(device)

    # その他ダミー
    f0_norm = torch.zeros_like(f0_tensor).to(device)
    loud_norm = torch.ones_like(f0_tensor).to(device) * 0.8 # 音量は大きめに

    model.eval()
    import IPython.display as ipd

    with torch.no_grad():
        for i, name in zip(test_ids, inst_names):
            inst_id = torch.tensor([i], device=device).long()

            # 生成
            audio = model(f0_tensor, f0_norm, loud_norm, inst_id)
            audio_np = audio.cpu().numpy().flatten()

            print(f"--- {name} ---")
            ipd.display(ipd.Audio(audio_np, rate=SAMPLE_RATE))

# 実行
listen_current_model(model)

▼▼▼ Epoch 10 時点の音色チェック ▼▼▼
--- Flute (Sine) ---


--- Violin (Saw) ---


--- Clarinet (Square) ---


In [32]:
def test_forced_generation_fixed():
    print("強制生成テスト（修正版）開始...")

    # 設定
    duration = 4.0
    # ★修正点1: 時間軸は「サンプル数」ではなく「フレーム数」で作る
    n_frames = int(duration * SAMPLE_RATE / HOP_LENGTH)

    # 0.0秒 〜 4.0秒 までを n_frames 分割
    t_frames = np.linspace(0, duration, n_frames, endpoint=False)

    # ★修正点2: f0もフレーム単位で作る
    # 1秒ごとに100Hz上がる (220, 320, 420...)
    f0_dummy = 220.0 + (np.floor(t_frames) * 100.0)
    f0_dummy = f0_dummy.astype(np.float32)

    # Tensor化 [1, T_frames, 1]
    f0_tensor = torch.from_numpy(f0_dummy).view(1, -1, 1).to(device)

    # 他のダミー入力もフレーム長に合わせる
    f0_norm = torch.zeros_like(f0_tensor).to(device)
    loud_norm = torch.ones_like(f0_tensor).to(device)
    inst_id = torch.tensor([0], device=device).long()

    # モデル設定（強制モードのまま）
    model.eval()
    with torch.no_grad():
        # これで出力は [1, n_frames * 64] になるはず
        audio_out = model(f0_tensor, f0_norm, loud_norm, inst_id)

    audio_np = audio_out.cpu().numpy().flatten()

    print(f"生成された音声の長さ: {len(audio_np)} サンプル ({len(audio_np)/SAMPLE_RATE:.2f}秒)")
    print("▼▼▼ 再生 ▼▼▼")

    import IPython.display as ipd
    ipd.display(ipd.Audio(audio_np, rate=SAMPLE_RATE))

# 実行
test_forced_generation_fixed()

強制生成テスト（修正版）開始...
生成された音声の長さ: 64000 サンプル (4.00秒)
▼▼▼ 再生 ▼▼▼


### 音の確認用コードの詳細

このコードは、学習が完了したDDSPモデルを使用して音声を生成し、その生成された音声と元の音声とを比較・確認するためのものです。

#### 1. 保存された重みファイルの検索とロード

-   **`list_of_files = glob.glob('ddsp_trumpet_*.pth')`**:
    -   `glob`モジュールを使用して、現在のディレクトリにある`ddsp_trumpet_`で始まるすべての`.pth`ファイル（モデルの学習済み重みが保存されているファイル）を検索します。
-   **エラーチェックと最新ファイルの選択**:
    -   `len(list_of_files) == 0`でファイルが見つからない場合はエラーメッセージを表示します。
    -   `latest_file = max(list_of_files, key=os.path.getctime)`: 複数の`.pth`ファイルが見つかった場合、`os.path.getctime`（最終更新時刻）をキーとして、最も新しい（最後に学習された）モデルの重みファイルを選択します。
-   **モデルの再構築と重みのロード**:
    -   `device = 'cuda' if torch.cuda.is_available() else 'cpu'`: GPUが利用可能であればGPUを、そうでなければCPUを使用するように設定します。
    -   `model = DDSP().to(device)`: 新しい`DDSP`モデルのインスタンスを生成し、適切なデバイスに移動します。
    -   `model.load_state_dict(torch.load(latest_file, map_location=device))`: 選択した`.pth`ファイルから学習済み重みをロードします。`map_location=device`は、保存時と異なるデバイスでロードする場合に、重みを現在のデバイスにマッピングするために必要です。
    -   `model.eval()`: モデルを評価モードに設定します。これにより、Batch NormalizationやDropoutなどの学習時のみ有効な層が無効になり、一貫した出力を得られます。

#### 2. 音声の生成と確認

-   **`with torch.no_grad():`**:
    -   推論時には勾配計算が不要なため、`torch.no_grad()`ブロックを使用します。これによりメモリ使用量が削減され、計算が高速化されます。
-   **データセットからのサンプル取得**:
    -   `if 'dataset' not in globals():`: もし`dataset`変数がまだ定義されていなければ、`SingleViolinDataset`を再生成します。
    -   `sample = dataset[0]`: データセットから最初のサンプルを取得します。このサンプルには、F0、ラウドネス、および元のオーディオ波形が含まれています。
-   **モデル入力の準備**:
    -   `f0_hz = sample['f0'].unsqueeze(0).to(device)`:
        -   `sample['f0']`はデータセットで抽出されたHz単位のF0です。
        -   `unsqueeze(0)`は、モデルが期待するバッチ次元（`[Batch, Time, Features]`）を追加します。
        -   `.to(device)`でデバイスに転送します。
    -   `f0_norm = sample['f0_norm'].unsqueeze(0).to(device)`:
        -   `sample['f0_norm']`はデータセットで正規化されたF0です。
        -   同様にバッチ次元を追加し、デバイスに転送します。
    -   `loudness_norm = sample['loudness_norm'].unsqueeze(0).to(device)`:
        -   `sample['loudness_norm']`はデータセットで正規化されたラウドネスです。
        -   同様にバッチ次元を追加し、デバイスに転送します。
-   **モデルによる音声生成**:
    -   `audio_out = model(f0_hz, f0_norm, loudness_norm)`: 準備された入力（物理量F0、正規化F0、正規化ラウドネス）をモデルに渡し、合成されたオーディオ (`audio_out`) を取得します。
-   **生成音声と元の音声の再生**:
    -   `ipd.display(ipd.Audio(audio_out.cpu().numpy().squeeze(), rate=16000))`: 生成されたオーディオをCPUに移動し、Numpy配列に変換し、不要な次元を削除（`squeeze()`）した後、`IPython.display.Audio`でColab上に再生します。
    -   `ipd.display(ipd.Audio(sample['audio'].numpy(), rate=16000))`: 元のオーディオも同様に再生し、生成された音声との比較を容易にします。

このコードを実行することで、学習されたDDSPモデルがどれだけ元の音声をうまく再現できているか、耳で確認することができます。

In [ ]:
# 推論時のデバッグ用コード例
# モデルのforwardを少し改造して、harmとnoiseを別々に返すモードを作るか、
# あるいは以下のように手動で内部の関数を呼ぶと確実です。

# (推論セルの中で)
with torch.no_grad():
    # 1. MLPを通す
    x = torch.cat([f0_norm, loudness_norm], dim=-1)
    x = model.mlp(x)
    x, _ = model.gru(x)

    # 2. パラメータ計算
    amp = torch.exp(model.dense_amp(x))
    harm_dist = F.softmax(model.dense_harm(x), dim=-1)
    noise_coeffs = torch.exp(model.dense_noise(x))

    # 3. 別々に合成
    audio_harm = model.harmonic_synth(f0, harm_dist, amp).squeeze()
    audio_noise = model.noise_synth(noise_coeffs).squeeze()

    # 合成
    audio_full = audio_harm + audio_noise

# 4. 保存して聴き比べ
ipd.display(ipd.Audio(audio_harm.cpu().numpy().squeeze(), rate=16000))
ipd.display(ipd.Audio(audio_noise.cpu().numpy().squeeze(), rate=16000))
ipd.display(ipd.Audio(audio_full.cpu().numpy().squeeze(), rate=16000))

NameError: name 'f0_norm' is not defined